In [ ]:
# task 1: list all the files in directory
# task 2: process all the files sequentially
# task 3: extract text from each file
# task 4: tokenize
# task 5: iterate over list of words and count unigrams,bigrams,trigrams and quadgrams 
# task 6: return a dictionary of unigrams,bigrams,trigrams and quadgrams

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import PyPDF2
from collections import Counter, defaultdict
import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize

In [ ]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

In [ ]:
my_text = "we were excited to study ML but as semester started we realised its very hard subject. We hope to study cool models but we were greeted with boring linear algebra on day one  and the assignments were so good and we were confident doing them but we failed to score good in all of them. chatgpt was such a loyal friend but we broke up just for ml to be specific ki kahi lihaycha te suchat sudha nahi pn maja yetiye ata timepass chaluye. "

In [ ]:
def extractNgrams(my_text):

    tokens = word_tokenize(my_text)

    unigrams_dict = {}
    bigrams_dict = {}
    trigrams_dict = {}
    quadgrams_dict = {}

    def cleanText(tokenList):
        goodTokens = [token.lower() for token in tokenList if token.isalnum()]
        return goodTokens

    tokens = cleanText(tokens)
    if len(tokens) < 4:
        return [[{}, {}, {}, {}]]
    print(len(tokens))
    for i in range(len(tokens)-3):
        token_1 = tokens[i]
        token_2 = tokens[i+1]
        token_3 = tokens[i+2]
        token_4 = tokens[i+3]

        unigram = token_1 
        bigram = token_1 + " " + token_2
        trigram = bigram + " " + token_3
        quadgram = trigram + " " + token_4

        if  unigram not in unigrams_dict:
            unigrams_dict[unigram] = 0
        unigrams_dict[unigram] += 1

        if bigram not in bigrams_dict:
            bigrams_dict[bigram] = 0
        bigrams_dict[bigram] += 1

        if trigram not in trigrams_dict:
            trigrams_dict[trigram] = 0
        trigrams_dict[trigram] += 1

        if quadgram not in quadgrams_dict:
            quadgrams_dict[quadgram] = 0
        quadgrams_dict[quadgram] += 1


    # ADDING REMAINING UNIGRAMS
    for token in tokens[len(tokens)-3:]:
        unigrams_dict[token] = unigrams_dict.get(token, 0) + 1

    # ADDING REMAING BIGRAMS
    for i in range(len(tokens)-3, len(tokens)-1):
        token_1 = tokens[i]
        token_2 = tokens[i+1]
        bigram = token_1 + " " + token_2
        if bigram not in bigrams_dict:
            bigrams_dict[bigram] = 0
        bigrams_dict[bigram] += 1

    token_1 = tokens[len(tokens)-3]
    token_2 = tokens[len(tokens)-2]
    token_3 = tokens[len(tokens)-1]
    trigram = token_1 + " " + token_2 + " " + token_3
    if trigram not in trigrams_dict:
        trigrams_dict[trigram] = 0
    trigrams_dict[trigram] += 1
    
    return [[unigrams_dict, bigrams_dict, trigrams_dict, quadgrams_dict]]


In [ ]:
pdf_folder = "batchspdf/"
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

pdf_texts = []

for pdf in pdf_files:
    text = extract_text_from_pdf(pdf)
    pdf_texts.append(text)

In [ ]:
ngrams_list = []
for pdf_text in pdf_texts:
    ngrams = extractNgrams(pdf_text)
    ngrams_list.extend(ngrams)

In [ ]:
print(len(ngrams_list))  # This will print the list of ngrams extracted from all PDFs/

In [ ]:
print(ngrams_list)  # This will print the list of ngrams extracted from all PDFs/

In [ ]:
# creating dataframe of unigrams with there frequency and no. of pdfs it occured in
unigrams_df = pd.DataFrame([unigram for ngram in ngrams_list for unigram in ngram[0].items()], columns=["unigram", "frequency"])
unigrams_df["num_pdfs"] = unigrams_df["unigram"].apply(lambda x: sum(1 for ngram in ngrams_list if x in ngram[0]))
unigrams_df["frequency_per_pdf"] = unigrams_df["frequency"] / unigrams_df["num_pdfs"]

In [ ]:
bigrams_df = pd.DataFrame([bigram for bigram in ngrams_list for bigram in bigram[1].items()],columns=["bigram", "frequency"])
bigrams_df["num_pdfs"] = bigrams_df["bigram"].apply(lambda x: sum(1 for ngram in ngrams_list if x in ngram[1]))
bigrams_df["frequency_per_pdf"] = bigrams_df["frequency"] / bigrams_df["num_pdfs"]

In [ ]:
trigrams_df = pd.DataFrame([trigram for trigram in ngrams_list for trigram in trigram[2].items()], columns=["trigram", "frequency"])
trigrams_df["num_pdfs"] = trigrams_df["trigram"].apply(lambda x: sum(1 for ngram in ngrams_list if x in ngram[2]))
trigrams_df["frequency_per_pdf"] = trigrams_df["frequency"] / trigrams_df["num_pdfs"]

In [ ]:
quadgrams_df = pd.DataFrame([quadgram for quadgram in ngrams_list for quadgram in quadgram[3].items()], columns=["quadgram", "frequency"])
quadgrams_df["num_pdfs"] = quadgrams_df["quadgram"].apply(lambda x: sum(1 for ngram in ngrams_list if x in ngram[3]))
quadgrams_df["frequency_per_pdf"] = quadgrams_df["frequency"] / quadgrams_df["num_pdfs"]


In [ ]:
def robust_zscore(data):
    median = data.median()
    iqrange = data.quantile(0.75) - data.quantile(0.25)
    return (data - median) / iqrange if iqrange else 0


In [ ]:
def zscore(data):
    mean = data.mean()
    std = data.std()
    return (data - mean) / std if std else 0

In [ ]:
def calculateMahalanobis(df):
    # Assuming df is already loaded
    numeric_df = df[["frequency", "num_pdfs"]]

    # Compute mean and covariance
    mean_vec = np.mean(numeric_df, axis=0)
    cov_matrix = np.cov(numeric_df, rowvar=False)
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # Function for Mahalanobis distance
    def mahalanobis(x, mean, inv_cov):
        diff = x - mean
        return np.sqrt(diff.T @ inv_cov @ diff)

    # Apply distance
    df["mahalanobis_distance"] = numeric_df.apply(
        lambda row: mahalanobis(row.values, mean_vec, inv_cov_matrix), axis=1
    )

    return df["mahalanobis_distance"]

In [ ]:
def MAD(df):
    data = np.array(df["frequency"])
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    z = (data - median) / mad 
    return mad

In [ ]:
# applying z score to unigrams on frquency
unigrams_df["z_score"] = zscore(unigrams_df["frequency"])
unigrams_df["robust_z_score"] = robust_zscore(unigrams_df["frequency"])
mahalanobis_distance = calculateMahalanobis(unigrams_df)

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(unigrams_df["frequency"], bins=100)
plt.title("Unigrams Frequency Distribution")
plt.xlabel("Frequency")
plt.ylabel("Count")
plt.show()

In [ ]:
unigrams_std_z_score = unigrams_df["z_score"].std()
left_threshold_z_score = -2.5 * unigrams_std_z_score
right_threshold_z_score = 2.5 * unigrams_std_z_score
plt.figure(figsize=(12, 6))
# plt.hist(unigrams_df["z_score"], bins=100)
sns.kdeplot(unigrams_df["z_score"], color='blue')
plt.axvline(x=0, color='r', linestyle='--')
plt.axvline(x=right_threshold_z_score, color='g', linestyle='--')
plt.axvline(x=left_threshold_z_score, color='g', linestyle='--')
plt.title("Unigrams Z-Score Distribution")
plt.xlabel("Z-Score")
plt.ylabel("Frequency")
plt.show()
print(unigrams_df["z_score"].min(),"<- min")
print(unigrams_df["z_score"].max(),"<- max")

In [ ]:
unigrams_outliers_zscore = unigrams_df[(unigrams_df["z_score"] > right_threshold_z_score) | (unigrams_df["z_score"] < left_threshold_z_score)]
unigrams_outliers_zscore["unigram"].to_csv("unigrams_outliers_zscore.csv", index=False)

In [ ]:
robust_iqr = unigrams_df["robust_z_score"].quantile(0.75) - unigrams_df["robust_z_score"].quantile(0.25)
right_threshold = unigrams_df["robust_z_score"].quantile(0.75) + 1.5 * robust_iqr
left_threshold = unigrams_df["robust_z_score"].quantile(0.25) - 1.5 * robust_iqr

plt.figure(figsize=(12, 6))
# plt.hist(unigrams_df["robust_z_score"], bins=100)
sns.kdeplot(unigrams_df["robust_z_score"],  color='blue')
plt.axvline(x=0, color='r', linestyle='--')
plt.axvline(x=right_threshold, color='g', linestyle='--')
plt.axvline(x=left_threshold, color='g', linestyle='--')
plt.title("Unigrams Robust Z-Score Distribution")
plt.xlabel("Robust Z-Score")
plt.ylabel("Frequency")
plt.show()
print(unigrams_df["robust_z_score"].min(),"<- min")
print(unigrams_df["robust_z_score"].max(),"<- max")

Export Outliers to CSV

In [ ]:
# exporting unigrams_df outliers with z score to csv
unigrams_outliers_robustzscore = unigrams_df[(unigrams_df["robust_z_score"] > right_threshold) | (unigrams_df["robust_z_score"] < left_threshold)]
unigrams_outliers_robustzscore["unigram"].to_csv("unigrams_outliers_robustzscore.csv", index=False)

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(unigrams_df["frequency"], unigrams_df["num_pdfs"])
plt.title("Unigrams Frequency vs Number of PDFs")
plt.xlabel("Frequency")
plt.ylabel("Number of PDFs")
plt.show()

In [ ]:
# knee-elbow cut
plt.figure(figsize=(24, 6))
sns.kdeplot(mahalanobis_distance, color='blue')
# plt.axvline(x=a, color='g', linestyle='--')
plt.axvline(x=2.0, color='g', linestyle='--')
plt.title("Unigrams Mahalanobis Distance Distribution")
plt.xlabel("Mahalanobis Distance")
plt.ylabel("Density")
plt.show()
print(mahalanobis_distance.shape)
# print(unigram for unigram in unigrams_df for unigrams_df["frequency"].max() in unigrams_df["frequency"])


In [ ]:
# exporting unigrams_df outliers with z score to csv
unigrams_outliers_mahalanobis = unigrams_df[(mahalanobis_distance > 2.0)]
unigrams_outliers_mahalanobis["unigram"].to_csv("unigrams_outliers_mahalanobis.csv", index=False)

REPEATING SAME FOR BIGRAMS

In [ ]:
bigrams_df["z_score"] = zscore(bigrams_df["frequency"])
bigrams_df["robust_z_score"] = robust_zscore(bigrams_df["frequency"])
bigrams_df["mahalanobis_distance"] = calculateMahalanobis(bigrams_df)

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(bigrams_df["z_score"], color='blue')
plt.axvline(x=0, color='r', linestyle='--')
plt.axvline(x=2.5, color='g', linestyle='--')
plt.axvline(x=-2.5, color='g', linestyle='--')
plt.xlabel("Z-Score")
plt.ylabel("density")
plt.show()

In [ ]:

bigrams_outlier_zscore = bigrams_df[(bigrams_df["z_score"]>2.5) | (bigrams_df["z_score"]<-2.5)]
bigrams_outlier_zscore["bigram"].to_csv("bigrams_outlier_zscore.csv", index=False)

In [ ]:
iqr = bigrams_df["robust_z_score"].quantile(0.75) - bigrams_df["robust_z_score"].quantile(0.25)
a = bigrams_df["robust_z_score"].quantile(0.75) + 1.5 * iqr
b = bigrams_df["robust_z_score"].quantile(0.25) - 1.5 * iqr
plt.figure(figsize=(24, 6))
sns.kdeplot(bigrams_df["robust_z_score"], color='blue')
plt.axvline(x=0, color='r', linestyle='--')
plt.axvline(x=a, color='g', linestyle='--')
plt.axvline(x=b, color='g', linestyle='--')
plt.xlabel("Robust Z-Score")
plt.ylabel("density")
plt.show()

In [ ]:

bigrams_outlier_robust_zscore = bigrams_df[(bigrams_df["robust_z_score"]<b) | (bigrams_df["robust_z_score"]>a)]
bigrams_outlier_robust_zscore["bigram"].to_csv("bigrams_outlier_robust_zscore.csv", index=False)

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(bigrams_df["mahalanobis_distance"], color='blue')
plt.axvline(x=1.4, color='r', linestyle='--')
plt.xlabel("Mahalanobis Distance")
plt.ylabel("density")
plt.show()

In [ ]:

bigrams_outlier_mahalanobis = bigrams_df[(bigrams_df["mahalanobis_distance"]>1.4)]
bigrams_outlier_mahalanobis["bigram"].to_csv("bigrams_outlier_mahalanobis.csv", index=False)

REPEATING SAME FOR TRIGRAMS

In [ ]:
trigrams_df["z_score"] = zscore(trigrams_df["frequency"])
trigrams_df["robust_z_score"] = robust_zscore(trigrams_df["frequency"])
print(trigrams_df["frequency"].quantile(0.75))
print(trigrams_df["frequency"].quantile(0.25))
trigrams_df["mahalanobis_distance"] = calculateMahalanobis(trigrams_df)
count = (trigrams_df["frequency"] == 1).sum()
print(count)

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(trigrams_df["frequency"], color='blue')
plt.axvline(x=trigrams_df["frequency"].quantile(0.75), color='r', linestyle='--')
plt.axvline(x=trigrams_df["frequency"].quantile(0.25), color='g', linestyle='--')
plt.xlabel("Frequency")
plt.ylabel("Density")
plt.title("Trigrams Frequency Distribution")
plt.show()

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(trigrams_df["z_score"], color='blue')
plt.axvline(x=0, color='r', linestyle='--')
plt.axvline(x=2.5, color='g', linestyle='--')
plt.axvline(x=-2.5, color='g', linestyle='--')
plt.xlabel("Z-Score")
plt.ylabel("density")
plt.show()

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(trigrams_df["mahalanobis_distance"], color='blue')
plt.axvline(x=1.0, color='g', linestyle='--')
plt.xlabel("Mahalanobis Distance")
plt.ylabel("Density")
plt.title("Trigrams Mahalanobis Distance Distribution")
plt.show()

REPEATING SAME FOR QUADGRAMS

In [ ]:
quadgrams_df["z_score"] = zscore(quadgrams_df["frequency"])
quadgrams_df["robust_z_score"] = robust_zscore(quadgrams_df["frequency"])
quadgrams_df["mahalanobis_distance"] = calculateMahalanobis(quadgrams_df)
# print(quadgrams_df[["frequency", "z_score", "robust_z_score", "mahalanobis_distance"]])

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(quadgrams_df["z_score"], color='blue')
plt.axvline(x=0, color='r', linestyle='--')
plt.axvline(x=2.5, color='g', linestyle='--')
plt.axvline(x=-2.5, color='g', linestyle='--')
plt.xlabel("Z-Score")
plt.ylabel("density")
plt.show()

In [ ]:
plt.figure(figsize=(24, 6))
sns.kdeplot(quadgrams_df["mahalanobis_distance"], color='blue')
plt.axvline(x=0.8, color='r', linestyle='--')
plt.xlabel("Mahalanobis Distance")
plt.ylabel("density")
plt.title("Quadgrams Mahalanobis Distance Distribution")
plt.show()